In [1]:
!pip install ta
#데이터셋을 만들어 주는 함수
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.layers import LSTM,Dropout,GRU
import os
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score

df = pd.read_csv("./[경기관련소비재]현대차.csv")
df_columns = ['Date','Open', 'High', 'Low', 'Close', 'Volume']
df.columns = df_columns
df = df.sort_values('Date')
df['Date']= df['Date'].astype('str')
from datetime import datetime
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date',inplace=True)

ma = [5,20,60,120]
for days in ma:
    df['ma_'+str(days)] = df['Close'].rolling(window = days).mean()
    df['vma_'+str(days)] = df['Volume'].rolling(window = days).mean()

df.dropna(inplace=True)


import ta

H, L, C, V = df['High'], df['Low'], df['Close'], df['Volume']

# df['bol_high'] = ta.volatility.bollinger_hband(C)
# df['bol_low']  = ta.volatility.bollinger_lband(C)
df['MFI'] = ta.volume.money_flow_index(
    high=H, low=L, close=C, volume=V, fillna=True)

df['ADI'] = ta.volume.acc_dist_index(
    high=H, low=L, close=C, volume=V, fillna=True)

df['OBV'] = ta.volume.on_balance_volume(close=C, volume=V, fillna=True)
df['CMF'] = ta.volume.chaikin_money_flow(
    high=H, low=L, close=C, volume=V, fillna=True)

df['FI'] = ta.volume.force_index(close=C, volume=V, fillna=True)
df['EOM, EMV'] = ta.volume.ease_of_movement(
    high=H, low=L, volume=V, fillna=True)

df['VPT'] = ta.volume.volume_price_trend(close=C, volume=V, fillna=True)
df['NVI'] = ta.volume.negative_volume_index(close=C, volume=V, fillna=True)
df['VMAP'] = ta.volume.volume_weighted_average_price(
    high=H, low=L, close=C, volume=V, fillna=True)

# Volatility
df['ATR'] = ta.volatility.average_true_range(
    high=H, low=L, close=C, fillna=True)
df['BHB'] = ta.volatility.bollinger_hband(close=C, fillna=True)
df['BLB'] = ta.volatility.bollinger_lband(close=C, fillna=True)
df['KCH'] = ta.volatility.keltner_channel_hband(
    high=H, low=L, close=C, fillna=True)
df['KCL'] = ta.volatility.keltner_channel_lband(
    high=H, low=L, close=C, fillna=True)
df['KCM'] = ta.volatility.keltner_channel_mband(
    high=H, low=L, close=C, fillna=True)
df['DCH'] = ta.volatility.donchian_channel_hband(
    high=H, low=L, close=C, fillna=True)
df['DCL'] = ta.volatility.donchian_channel_lband(
    high=H, low=L, close=C, fillna=True)
df['DCM'] = ta.volatility.donchian_channel_mband(
    high=H, low=L, close=C, fillna=True)
df['UI'] = ta.volatility.ulcer_index(close=C, fillna=True)
# Trend
df['SMA'] = ta.trend.sma_indicator(close=C, fillna=True)
df['EMA'] = ta.trend.ema_indicator(close=C, fillna=True)
df['WMA'] = ta.trend.wma_indicator(close=C, fillna=True)
df['MACD'] = ta.trend.macd(close=C, fillna=True)
df['ADX'] = ta.trend.adx(high=H, low=L, close=C, fillna=True)
df['-VI'] = ta.trend.vortex_indicator_neg(
    high=H, low=L, close=C, fillna=True)
df['+VI'] = ta.trend.vortex_indicator_pos(
    high=H, low=L, close=C, fillna=True)
df['TRIX'] = ta.trend.trix(close=C, fillna=True)
df['MI'] = ta.trend.mass_index(high=H, low=L, fillna=True)
df['CCI'] = ta.trend.cci(high=H, low=L, close=C, fillna=True)
df['DPO'] = ta.trend.dpo(close=C, fillna=True)
df['KST'] = ta.trend.kst(close=C, fillna=True)
df['Ichimoku'] = ta.trend.ichimoku_a(high=H, low=L, fillna=True)
df['Parabolic SAR'] = ta.trend.psar_down(
    high=H, low=L, close=C, fillna=True)
df['STC'] = ta.trend.stc(close=C, fillna=True)
# Momentum
df['RSI'] = ta.momentum.rsi(close=C, fillna=True)
df['SRSI'] = ta.momentum.stochrsi(close=C, fillna=True)
df['TSI'] = ta.momentum.tsi(close=C, fillna=True)
df['UO'] = ta.momentum.ultimate_oscillator(
    high=H, low=L, close=C, fillna=True)
df['SR'] = ta.momentum.stoch(close=C, high=H, low=L, fillna=True)
df['WR'] = ta.momentum.williams_r(high=H, low=L, close=C, fillna=True)
df['AO'] = ta.momentum.awesome_oscillator(high=H, low=L, fillna=True)
df['KAMA'] = ta.momentum.kama(close=C, fillna=True)
df['ROC'] = ta.momentum.roc(close=C, fillna=True)
df['PPO'] = ta.momentum.ppo(close=C, fillna=True)
df['PVO'] = ta.momentum.pvo(volume=V, fillna=True)


from sklearn.preprocessing import MinMaxScaler

# 피처값 스케일링
scaler = MinMaxScaler()

scaled_df = scaler.fit_transform(df.drop(['Close'], axis=1))
scaled_df = pd.DataFrame(scaled_df, columns = df.drop(['Close'], axis=1).columns)

scaler1 = MinMaxScaler()

temp = scaler1.fit_transform(df['Close'].values.reshape(-1,1))

scaled_df.insert(3, 'Close',temp)
scaled_df

df = scaled_df



import numpy as np

def make_dataset(feature, label, window_size = 20):
  
  feature_list = []
  label_list = []
  
  for i in range(len(feature) - window_size) :
    feature_list.append(feature.iloc[i:i+window_size])
    label_list.append(label.iloc[i+window_size])
  
  return np.array(feature_list), np.array(label_list)

WINDOW_SIZE=20

result = []
predict = []


####최근 20개의 데이터만
pred_feature = df[-WINDOW_SIZE :]
pred_feature = pred_feature.to_numpy()

pred_feature = pred_feature.reshape((1,20,-1))
# pred_feature.shape

import random as rn
import tensorflow as tf


for i in range(1, len(df)//1000+2):

        # 데이터 불러오기
        df_copy = df
     
        # 원하는 크기로 데이터 자르기
        df_copy = df_copy[-i*1000:]
        # 피처값, 타켓 스케일링
        
        TEST_SIZE = int(len(df_copy) * 0.7)
        
        train = df_copy[:TEST_SIZE]
        test = df_copy[TEST_SIZE:]

##200일의 데이터를 얼마나 잘 예측하느냐
        
        # feature_cols = df.drop('Close', axis =1).columns
        feature_cols = df.columns
        label_cols = ['Close']

        train_feature = train[feature_cols]
        train_label = train[label_cols]

        train_feature, train_label = make_dataset(train_feature, train_label,20)

        from sklearn.model_selection import train_test_split
        x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2, shuffle = False)
        
        
        test_feature = test[feature_cols]
        test_label = test[label_cols]
        test_feature , test_label = make_dataset(test_feature, test_label,20)
        
        
        from keras.layers.normalization import batch_normalization

        model = Sequential()
        model.add(GRU(64, 
                      input_shape=(train_feature.shape[1], train_feature.shape[2]), 
                      return_sequences = True,
                      activation='relu'))
        model.add(GRU(64, return_sequences=False, stateful=False))
        model.add(Dropout(0.2))
        model.add(Dense(1))
      
        
        model.compile(loss='mean_squared_error', optimizer='adam')

        early_stop = EarlyStopping(monitor='val_loss', patience=10)

        import os
        from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

        model_path = 'model'
        filename = os.path.join(model_path, 'tmp_checkpoint.h5')
        checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

        history = model.fit(x_train, y_train, 
                                            epochs=100, 
                                            batch_size=16,
                                            validation_data=(x_valid, y_valid), 
                                            callbacks=[early_stop, checkpoint])


        pred = model.predict(test_feature)
        # rescaleing 작업
        rescaled_y_true = scaler1.inverse_transform(np.array(test_label).reshape(-1, 1))
        rescaled_pred = scaler1.inverse_transform(np.array(pred).reshape(-1,1))
        
        
        # 평가지표(RMSE) 계산
        RMSE = np.sqrt(mean_squared_error(rescaled_y_true, rescaled_pred))
        result.append(RMSE)
        print(f"{i * 1000}길이의 데이터 적용 완료\n 길이: {i * 1000}, RMSE:{RMSE}")
        print(result)
        
        

        new_pred = model.predict(pred_feature)
        rescaled_newpred = scaler1.inverse_transform(np.array(new_pred).reshape(-1,1))
        predict.append(rescaled_newpred)



C:\Users\gram\anaconda3\lib\site-packages\ta\trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
C:\Users\gram\anaconda3\lib\site-packages\ta\trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


Epoch 1/100
32/34 [===========================>..] - ETA: 0s - loss: 0.0239
Epoch 1: val_loss improved from inf to 0.05278, saving model to model\tmp_checkpoint.h5
34/34 [==============================] - 8s 72ms/step - loss: 0.0227 - val_loss: 0.0528
Epoch 2/100
32/34 [===========================>..] - ETA: 0s - loss: 0.0053
Epoch 2: val_loss improved from 0.05278 to 0.02892, saving model to model\tmp_checkpoint.h5
34/34 [==============================] - 1s 26ms/step - loss: 0.0053 - val_loss: 0.0289
Epoch 3/100
34/34 [==============================] - ETA: 0s - loss: 0.0030
Epoch 3: val_loss improved from 0.02892 to 0.01996, saving model to model\tmp_checkpoint.h5
34/34 [==============================] - 1s 27ms/step - loss: 0.0030 - val_loss: 0.0200
Epoch 4/100
34/34 [==============================] - ETA: 0s - loss: 0.0026
Epoch 4: val_loss improved from 0.01996 to 0.01721, saving model to model\tmp_checkpoint.h5
34/34 [==============================] - 1s 27ms/step - loss: 0.0026

69/69 [==============================] - 2s 22ms/step - loss: 0.0018 - val_loss: 4.2132e-04
Epoch 11/100
69/69 [==============================] - ETA: 0s - loss: 0.0017
Epoch 11: val_loss improved from 0.00022 to 0.00013, saving model to model\tmp_checkpoint.h5
69/69 [==============================] - 2s 24ms/step - loss: 0.0017 - val_loss: 1.2875e-04
Epoch 12/100
68/69 [============================>.] - ETA: 0s - loss: 0.0017
Epoch 12: val_loss did not improve from 0.00013
69/69 [==============================] - 2s 23ms/step - loss: 0.0017 - val_loss: 4.7679e-04
Epoch 13/100
67/69 [============================>.] - ETA: 0s - loss: 0.0017
Epoch 13: val_loss did not improve from 0.00013
69/69 [==============================] - 2s 22ms/step - loss: 0.0017 - val_loss: 5.8128e-04
Epoch 14/100
69/69 [==============================] - ETA: 0s - loss: 0.0016
Epoch 14: val_loss did not improve from 0.00013
69/69 [==============================] - 1s 21ms/step - loss: 0.0016 - val_loss: 0.0017

104/104 [==============================] - 2s 23ms/step - loss: 0.0019 - val_loss: 1.4487e-04
Epoch 25/100
103/104 [============================>.] - ETA: 0s - loss: 0.0017
Epoch 25: val_loss did not improve from 0.00014
104/104 [==============================] - 2s 23ms/step - loss: 0.0017 - val_loss: 1.6324e-04
Epoch 26/100
104/104 [==============================] - ETA: 0s - loss: 0.0016
Epoch 26: val_loss did not improve from 0.00014
104/104 [==============================] - 2s 22ms/step - loss: 0.0016 - val_loss: 2.2830e-04
Epoch 27/100
103/104 [============================>.] - ETA: 0s - loss: 0.0015
Epoch 27: val_loss improved from 0.00014 to 0.00014, saving model to model\tmp_checkpoint.h5
104/104 [==============================] - 2s 22ms/step - loss: 0.0015 - val_loss: 1.3794e-04
Epoch 28/100
103/104 [============================>.] - ETA: 0s - loss: 0.0016
Epoch 28: val_loss did not improve from 0.00014
104/104 [==============================] - 2s 24ms/step - loss: 0.0016 

Epoch 23/100
137/139 [============================>.] - ETA: 0s - loss: 0.0014
Epoch 23: val_loss did not improve from 0.00018
139/139 [==============================] - 2s 17ms/step - loss: 0.0014 - val_loss: 1.9620e-04
Epoch 24/100
136/139 [============================>.] - ETA: 0s - loss: 0.0013
Epoch 24: val_loss did not improve from 0.00018
139/139 [==============================] - 2s 17ms/step - loss: 0.0013 - val_loss: 2.0868e-04
Epoch 25/100
137/139 [============================>.] - ETA: 0s - loss: 0.0012
Epoch 25: val_loss did not improve from 0.00018
139/139 [==============================] - 2s 17ms/step - loss: 0.0012 - val_loss: 2.6545e-04
Epoch 26/100
137/139 [============================>.] - ETA: 0s - loss: 0.0012
Epoch 26: val_loss did not improve from 0.00018
139/139 [==============================] - 2s 16ms/step - loss: 0.0012 - val_loss: 3.2283e-04
4000길이의 데이터 적용 완료
 길이: 4000, RMSE:5789.1360290692
[10841.073735585862, 9452.703892897685, 7640.322476356367, 5789.13

...

243/244 [============================>.] - ETA: 0s - loss: 0.0045
Epoch 1: val_loss improved from inf to 0.01258, saving model to model\tmp_checkpoint.h5
244/244 [==============================] - 8s 21ms/step - loss: 0.0045 - val_loss: 0.0126
Epoch 2/100
243/244 [============================>.] - ETA: 0s - loss: 5.7718e-04
Epoch 2: val_loss improved from 0.01258 to 0.00750, saving model to model\tmp_checkpoint.h5
244/244 [==============================] - 4s 18ms/step - loss: 5.7658e-04 - val_loss: 0.0075
Epoch 3/100
243/244 [============================>.] - ETA: 0s - loss: 4.2617e-04
Epoch 3: val_loss improved from 0.00750 to 0.00410, saving model to model\tmp_checkpoint.h5
244/244 [==============================] - 4s 15ms/step - loss: 4.2534e-04 - val_loss: 0.0041
Epoch 4/100
242/244 [============================>.] - ETA: 0s - loss: 3.2792e-04
Epoch 4: val_loss improved from 0.00410 to 0.00334, saving model to model\tmp_checkpoint.h5
244/244 [==============================] - 4s 

244/244 [==============================] - ETA: 0s - loss: 8.9909e-05
Epoch 34: val_loss did not improve from 0.00038
244/244 [==============================] - 6s 25ms/step - loss: 8.9909e-05 - val_loss: 9.4469e-04
Epoch 35/100
243/244 [============================>.] - ETA: 0s - loss: 8.0739e-05
Epoch 35: val_loss did not improve from 0.00038
244/244 [==============================] - 6s 25ms/step - loss: 8.0585e-05 - val_loss: 5.5583e-04
Epoch 36/100
243/244 [============================>.] - ETA: 0s - loss: 9.0853e-05
Epoch 36: val_loss did not improve from 0.00038
244/244 [==============================] - 4s 18ms/step - loss: 9.0708e-05 - val_loss: 4.8460e-04
Epoch 37/100
242/244 [============================>.] - ETA: 0s - loss: 8.3309e-05
Epoch 37: val_loss improved from 0.00038 to 0.00038, saving model to model\tmp_checkpoint.h5
244/244 [==============================] - 3s 14ms/step - loss: 8.3726e-05 - val_loss: 3.7901e-04
Epoch 38/100
241/244 [============================>.

276/279 [============================>.] - ETA: 0s - loss: 8.4913e-05
Epoch 16: val_loss did not improve from 0.00139
279/279 [==============================] - 4s 14ms/step - loss: 8.4674e-05 - val_loss: 0.0020
Epoch 17/100
278/279 [============================>.] - ETA: 0s - loss: 8.8677e-05
Epoch 17: val_loss did not improve from 0.00139
279/279 [==============================] - 5s 17ms/step - loss: 8.8831e-05 - val_loss: 0.0019
Epoch 18/100
275/279 [============================>.] - ETA: 0s - loss: 7.4329e-05
Epoch 18: val_loss did not improve from 0.00139
279/279 [==============================] - 4s 15ms/step - loss: 7.4109e-05 - val_loss: 0.0025
Epoch 19/100
276/279 [============================>.] - ETA: 0s - loss: 7.3289e-05
Epoch 19: val_loss improved from 0.00139 to 0.00135, saving model to model\tmp_checkpoint.h5
279/279 [==============================] - 4s 14ms/step - loss: 7.3197e-05 - val_loss: 0.0014
Epoch 20/100
279/279 [==============================] - ETA: 0s - lo

313/314 [============================>.] - ETA: 0s - loss: 4.7590e-05
Epoch 15: val_loss improved from 0.00020 to 0.00018, saving model to model\tmp_checkpoint.h5
314/314 [==============================] - 4s 14ms/step - loss: 4.7789e-05 - val_loss: 1.8302e-04
Epoch 16/100
310/314 [============================>.] - ETA: 0s - loss: 4.3132e-05
Epoch 16: val_loss did not improve from 0.00018
314/314 [==============================] - 4s 14ms/step - loss: 4.3416e-05 - val_loss: 3.8843e-04
Epoch 17/100
311/314 [============================>.] - ETA: 0s - loss: 4.9229e-05
Epoch 17: val_loss did not improve from 0.00018
314/314 [==============================] - 4s 14ms/step - loss: 4.9026e-05 - val_loss: 6.1137e-04
Epoch 18/100
312/314 [============================>.] - ETA: 0s - loss: 4.2913e-05
Epoch 18: val_loss did not improve from 0.00018
314/314 [==============================] - 4s 14ms/step - loss: 4.2809e-05 - val_loss: 8.7566e-04
Epoch 19/100
312/314 [============================>.

341/341 [==============================] - ETA: 0s - loss: 2.7273e-05
Epoch 14: val_loss did not improve from 0.00047
341/341 [==============================] - 9s 26ms/step - loss: 2.7273e-05 - val_loss: 9.1948e-04
Epoch 15/100
341/341 [==============================] - ETA: 0s - loss: 2.5492e-05
Epoch 15: val_loss did not improve from 0.00047
341/341 [==============================] - 8s 23ms/step - loss: 2.5492e-05 - val_loss: 0.0014
Epoch 16/100
339/341 [============================>.] - ETA: 0s - loss: 2.6479e-05
Epoch 16: val_loss did not improve from 0.00047
341/341 [==============================] - 7s 19ms/step - loss: 2.6425e-05 - val_loss: 5.7377e-04
Epoch 17/100
339/341 [============================>.] - ETA: 0s - loss: 2.4743e-05
Epoch 17: val_loss did not improve from 0.00047
341/341 [==============================] - 6s 19ms/step - loss: 2.4682e-05 - val_loss: 0.0012
Epoch 18/100
339/341 [============================>.] - ETA: 0s - loss: 2.3921e-05
Epoch 18: val_loss did

In [2]:
print(predict)

[array([[180014.52]], dtype=float32), array([[193847.53]], dtype=float32), array([[188753.]], dtype=float32), array([[195478.86]], dtype=float32), array([[193403.25]], dtype=float32), array([[196217.23]], dtype=float32), array([[195103.83]], dtype=float32), array([[200225.23]], dtype=float32), array([[205593.67]], dtype=float32), array([[188420.38]], dtype=float32)]
